In [1]:
from __future__                           import print_function
%matplotlib inline

import re
import os
import sys
import h5py

import matplotlib.pyplot                  as plt
import numpy                              as np
import random                             as rd
import tables                             as tb
import tensorflow                         as tf

from glob                                 import glob
from matplotlib.patches                   import Ellipse


# Keras imports
import keras.backend.tensorflow_backend   as   K

from keras                                import regularizers
from keras                                import callbacks

from keras.models                         import Model, load_model

from keras.layers                         import Input, Dense, MaxPooling3D, AveragePooling3D
from keras.layers                         import Conv3D, Conv2D, AveragePooling2D, Activation, Dropout, ZeroPadding3D
from keras.layers                         import Add
from keras.layers                         import Flatten, BatchNormalization

from keras.layers.normalization           import BatchNormalization
from keras.layers.convolutional           import Convolution2D, MaxPooling2D
from keras.layers.core                    import Flatten

from keras.optimizers                     import SGD, Adam, Nadam, Adadelta 

from keras.regularizers                   import l2, l1
from keras.initializers                   import RandomNormal

from keras.utils.layer_utils              import print_summary

#for keras example MLP
from keras.models import Sequential


os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

Using TensorFlow backend.


### Create a combined file from several .npz files

In [ ]:
npz_dir = "/home/amenkara/PETALO_dnns/dnn/"

# input/output files

files = glob(npz_dir + '*.npz')

files = sorted(files)
print(files)

out_file = "combined_file_15_january.npz"

X_    = []
Compt_= []
Ev_   = []


for f in files:

    fn = np.load(f)
    
    if(len(X_) == 0):
        X_     = fn['images']
        Compt_ =  fn['labels']
        Ev_    =  fn['event_number']
    else:
        X_     = np.concatenate((X_,     fn['images'])      )
        Compt_ = np.concatenate((Compt_, fn['labels'])      ) 
        Ev_    = np.concatenate((Ev_,    fn['event_number']))
    
print("Saving combined file")
np.savez(out_file, X_=X_, Compt_=Compt_, Ev_=Ev_)

### Load the data from the dataset

In [2]:
out_file = "combined_file_15_january.npz"

In [3]:
data = np.load(out_file)
X_, Compt_, Ev_ = data['X_'],data['Compt_'], data['Ev_']

In [4]:
all_compton_events_labels        =  []
all_compton_events_images        =  []
all_compton_events_id            =  []

all_photoelectric_events_labels  =  []
all_photoelectric_events_images  =  []
all_photoelectric_events_id      =  []

In [5]:
#Split the events into photoelectric and Compton.

for i,event in enumerate(Ev_):
    if Compt_[i]== 0:
        all_photoelectric_events_labels.append(Compt_[i])
        all_photoelectric_events_images.append(X_[i])
        all_photoelectric_events_id.append(event)
    if Compt_[i]==1:
        all_compton_events_labels.append(Compt_[i])
        all_compton_events_images.append(X_[i])
        all_compton_events_id.append(event)

In [6]:
#then cut the longest in order it to be of the same lenght.
cut_compton_events_labels =  all_compton_events_labels[: len(all_photoelectric_events_id)]
cut_compton_events_images =  all_compton_events_images[: len(all_photoelectric_events_id)]
cut_compton_events_id     =  all_compton_events_id[: len(all_photoelectric_events_id)]

In [7]:
X_train     = []
Compt_train = []
Ev_train    = []

In [8]:
X_train_list     = all_photoelectric_events_images + cut_compton_events_images
Compt_train_list = all_photoelectric_events_labels + cut_compton_events_labels
Ev_train_list    = all_photoelectric_events_id     + cut_compton_events_id

In [9]:
X_= np.array(X_train_list)
Compt_= np.array(Compt_train_list)

### Separate train, validation and test set

In [10]:
train_cut = int(0.8*len(X_))
val_cut   = int(0.9*len(X_))

X_train      = X_[:train_cut]
X_val        = X_[train_cut:val_cut] 
X_test       = X_[val_cut:]

Compt_train  =  Compt_[:train_cut]
Compt_val    =  Compt_[train_cut:val_cut]
Compt_test   =  Compt_[val_cut:]

### Define the model

In [11]:
K.clear_session()

In [ ]:
def model_CNN(inputs):
    initial = "random_uniform"
    #el 32 implica que hay 32 filtros de 5x5
    cinputs = Conv2D(96, (5, 5), padding='same', strides=(2, 2), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(inputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = Conv2D(384, (2, 2), padding='same', strides=(1, 1), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01), activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    
    f1 = Flatten()(cinputs)
    f1 = Dense(units=64, activation='relu', kernel_initializer=initial)(f1)
    #f1 = Dropout(.2)(f1)

    inc_output = Dense(units=1, activation='sigmoid', kernel_initializer=initial)(f1)
    
    model = Model(inputs, inc_output)
    
    model.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.001), metrics=['accuracy'])
    
    return model

In [ ]:
def model_test(inputs):
    cinputs = Conv2D(96, (5, 5))(inputs)
    cinputs = Dense(32)(cinputs)
    internal = Dense(32)(cinputs)
    internal = Flatten()(internal)
    output = Dense(1, activation='sigmoid')(internal)
    
    model = Model(inputs, output)
    
    model.compile(loss='binary_crossentropy', optimizer=Nadam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004))
    return model

In [13]:
def model_VGG_like_convnet(inputs_):
    model = Sequential()
    # input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
    # this applies 32 convolution filters of size 3x3 each.
    cinputs = Conv2D(32, (3, 3), activation='relu')(inputs_)
    cinputs = Conv2D(32, (3, 3), activation='relu')(cinputs)
    cinputs = MaxPooling2D(pool_size=(2, 2))(cinputs)
    cinputs = Dropout(0.25)

    #model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    cinputs = Flatten(cinputs)
    cinputs = Dense(256, activation='relu')(cinputs)
    cinputs = Dropout(0.5)(cinputs)
    cinputs = Dense(10, activation='softmax')(cinputs)
    
    f1 = Flatten()(cinputs)
    inc_output = Dense(units=1, activation='sigmoid', kernel_initializer=initial)(cinputs)

    model = Model(inputs, inc_output)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    return model

In [14]:
dim     = 30

input_  = Input((dim,dim,1))
model   = model_VGG_like_convnet(input_)

AttributeError: 'Dropout' object has no attribute 'lower'

In [ ]:
model.summary()

In [ ]:
tboard = callbacks.TensorBoard(log_dir='./logs/Keras_VGG_lr_0_0001', histogram_freq=0, write_graph=True, write_images=False)

### Training

In [ ]:
model.fit(X_train, Compt_train, batch_size=180, epochs=200, shuffle=True, callbacks=[tboard],verbose=1, validation_data=(X_val, Compt_val))

In [ ]:
#line from keras docu.
score = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
model.save('model_conv2d_16_january_last_layer_activation_is_relu.h5')

In [ ]:
loaded_model = load_model('model_conv2d_16_january.h5')

In [ ]:
prediction = model.predict(X_test, batch_size=18, verbose=1)

In [ ]:
print(prediction[0:300])

In [ ]:
#Cut is the accuracy. 
def plot_result(cut):
elements = []
for element in prediction:
    if element > cut:
        elements.append(0)
    else:
        elements.append(1)
        
    aciertos = 0
    total = len(elements)
    for i in range(total):
        if elements[i]== Compt_test[i]:
            aciertos += 1
        
    print(aciertos)    
    return aciertos/total

In [ ]:
matrix = []
for i in range(500, 1000):
    matrix.append([i/1000., plot_result(i/1000.)])
        

In [ ]:
np.transpose(matrix)[1]

In [ ]:
plt.plot(np.transpose(matrix)[0][:], np.transpose(matrix)[1][:])